In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

##### SAS Data import

In [2]:
print(os.getcwd())

c:\Users\Owner\Desktop\윤태준\고용패널\3월 분석


In [5]:
df = pd.read_sas('data.sas7bdat', format='sas7bdat')
df.head()

c:\Users\Owner\anaconda3\envs\'pannel'\lib\site-packages\pandas\core\tools\datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
c:\Users\Owner\anaconda3\envs\'pannel'\lib\site-packages\pandas\core\tools\datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
c:\Users\Owner\anaconda3\envs\'pannel'\lib\site-packages\pandas\core\tools\datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


,pid,majorcat,province,area,sex,age,a003,a144,a146,d134,...,score,q001G,p001G,provinceG,major_sameG,major_helpG,majorcatG,q015G,q016G,q017G
0,100166.0,1.0,2.0,8.0,2.0,22.500000,NaN,NaN,NaN,3.0,...,2.093023,2.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,3.0
1,100191.0,1.0,4.0,7.0,2.0,26.916667,NaN,NaN,NaN,3.0,...,3.000000,3.0,1.0,2.0,2.0,3.0,1.0,2.0,3.0,3.0
2,100212.0,1.0,2.0,8.0,2.0,24.666667,NaN,NaN,NaN,1.0,...,2.093023,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0
3,100221.0,1.0,2.0,8.0,2.0,25.250000,NaN,NaN,NaN,1.0,...,3.000000,3.0,1.0,1.0,1.0,2.0,1.0,3.0,3.0,3.0
4,100232.0,1.0,3.0,11.0,1.0,25.250000,2.0,2.0,2.0,NaN,...,3.000000,3.0,1.0,2.0,1.0,1.0,1.0,3.0,3.0,3.0


모델 Logistic Regression
1. 인구통계학적 특성
2. 전공 연관성, 대학 지역, 학점, 재학 중 경험 일자리(완전 대학 관련)
3. 삶의 만족도(개인, 관계, 소속 집단)
    3-1 삶의 만족도 평균
4. full model
---------------------------------------------------
- 근속 종속변수 : :LongWork, LongWork2가 존재
    - LongWork2가 2년 이상 일한 사람 
    - 2년 ~ 3년 사이 기간을 분석하기로 했으니 Target : LongWork2로 지정해서 사용

<h6> 1번 인구통계학적특징 로지스틱 </h6>

In [6]:
model_human = sm.Logit.from_formula('''LongWork2 ~ age + C(sex) + C(q001G) + C(p001G)''', df).fit()
print(model_human.summary())
print(f'모델 AIC:{model_human.aic}')
print('\n')
print(f'오즈비{np.exp(model_human.params)}')
print('\n')

Optimization terminated successfully.
         Current function value: 0.544678
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5802
Method:                           MLE   Df Model:                            5
Date:                Thu, 30 Mar 2023   Pseudo R-squ.:                 0.06110
Time:                        03:35:01   Log-Likelihood:                -3163.5
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                 8.731e-87
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -8.9148      0.575    -15.496      0.000     -10.042      -7.787
C(sex)[T.2.0] 

###### 2번 대학 생활 로지스틱 회귀모형

In [8]:
model_college = sm.Logit.from_formula('''LongWork2 ~ C(provinceG) + C(major_sameG) + C(major_helpG) 
    + C(provinceG) + C(f073) + C(h001)''', df).fit()
print(model_college.summary())
print(f'모델 AIC:{model_college.aic}')
print('\n')
print(f'오즈비{np.exp(model_college.params)}')
print('\n')

Optimization terminated successfully.
         Current function value: 0.556455
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5798
Method:                           MLE   Df Model:                            9
Date:                Thu, 30 Mar 2023   Pseudo R-squ.:                 0.04080
Time:                        03:35:38   Log-Likelihood:                -3231.9
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                 5.333e-54
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -2.0191      0.183    -11.004      0.000      -2.379      -1.

###### 3번 삶의 만족도 로지스틱 

In [7]:
model_satisfaction = sm.Logit.from_formula('''LongWork2 ~ C(q015) + C(q016)
    + C(q017) ''', df).fit()
print(model_satisfaction.summary())
print(f'모델 AIC:{model_satisfaction.aic}')
print('\n')
print(f'오즈비{np.exp(model_satisfaction.params)}')
print('\n')

Optimization terminated successfully.
         Current function value: 0.570363
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5789
Method:                           MLE   Df Model:                           18
Date:                Thu, 30 Mar 2023   Pseudo R-squ.:                 0.01683
Time:                        03:35:23   Log-Likelihood:                -3312.7
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                 7.356e-16
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -2.0327      0.358     -5.671      0.000      -2.735      -1.330
C(q015)[T.2.0]   